In [ ]:
%env OPENAI_API_KEY = api_key

In [ ]:
# pip install --upgrade openai

In [ ]:
import glob
import json
import os
import re
import random
import time
import torch
from transformers import RobertaModel, RobertaTokenizer
import numpy as np

from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
cd '/home/azmain/alljavajsons'

In [ ]:
inputFiles = []
correctOutputFiles = []
    
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

for file in sorted(glob.glob("*.java.json"), key=numericalSort):
    inputFiles.append(file)

for file in sorted(glob.glob("*.benchmark_log.json"), key=numericalSort):
    correctOutputFiles.append(file)

print(inputFiles)
print('\n\n\n')
print(correctOutputFiles)

In [ ]:
def get_codes(inputFiles):
    codes = []
    for code in inputFiles:
        codes.append(str(json.load(open(code))['originalContent']))

    return codes

In [ ]:
def get_correct_outputs(correctOutputFiles):
    correct_outputs = []
    for output in correctOutputFiles:
        correct_output_list = json.load(open(output))['total_imports']
        correct_output_list = ["import "+i+";" for i in correct_output_list]
        correct_outputs.append(correct_output_list)
    
    for import_lines in correct_outputs:
        if "import gen.R;" in import_lines:
            import_lines.remove("import gen.R;")

    return correct_outputs

In [ ]:
def get_dataset(codes, correct_outputs):
    dataset = {
        "codes": codes,
        "correct_outputs": correct_outputs
    }
    return dataset

In [ ]:
def get_test_examples_and_y_true(dataset):
    sample_list = []

    for i in range(0, len(dataset["codes"])):
        sample_list.append(dict(codes=dataset["codes"][i], correct_outputs=dataset["correct_outputs"][i]))
    
    # print(sample_list)
    
    test_examples = [(example["codes"], example["correct_outputs"]) for example in sample_list]
    y_true = [correct_outputs for _, correct_outputs in test_examples]
    
    return test_examples, y_true

### All Android Classes

In [ ]:
print("Total Android Codes: {}\n".format(len(inputFiles[:50])))
print(inputFiles[:50])

android_codes = get_codes(inputFiles[:50])
# print(android_codes)

android_correct_outputs = get_correct_outputs(correctOutputFiles[:50])
# print(android_correct_outputs)

android_dataset = get_dataset(android_codes, android_correct_outputs)
# print(android_dataset)

android_test_examples, android_y_true = get_test_examples_and_y_true(android_dataset)
# print(android_test_examples)

### All JDK Classes

In [ ]:
print("Total JDK Codes: {}\n".format(len(inputFiles[50:73])))
print(inputFiles[50:73])

jdk_codes = get_codes(inputFiles[50:73])
# print(jdk_codes)

jdk_correct_outputs = get_correct_outputs(correctOutputFiles[50:73])
# print(jdk_correct_outputs)

jdk_dataset = get_dataset(jdk_codes, jdk_correct_outputs)
# print(jdk_dataset)

jdk_test_examples, jdk_y_true = get_test_examples_and_y_true(jdk_dataset)
# print(jdk_test_examples)

### All Hibernate Classes

In [ ]:
print("Total Hibernate Codes: {}\n".format(len(inputFiles[73:74] + inputFiles[174:224])))
print(inputFiles[73:74] + inputFiles[174:224])

hibernate_codes = get_codes(inputFiles[73:74] + inputFiles[174:224])
# print(hibernate_codes)

hibernate_correct_outputs = get_correct_outputs(correctOutputFiles[73:74] + correctOutputFiles[174:224])
# print(hibernate_correct_outputs)

hibernate_dataset = get_dataset(hibernate_codes, hibernate_correct_outputs)
# print(hibernate_dataset)

hibernate_test_examples, hibernate_y_true = get_test_examples_and_y_true(hibernate_dataset)
# print(hibernate_test_examples)

### All JodaTime Classes

In [ ]:
print("Total JodaTime Codes: {}\n".format(len(inputFiles[74:124])))
print(inputFiles[74:124])

jodatime_codes = get_codes(inputFiles[74:124])
# print(jodatime_codes)

jodatime_correct_outputs = get_correct_outputs(correctOutputFiles[74:124])
# print(jodatime_correct_outputs)

jodatime_dataset = get_dataset(jodatime_codes, jodatime_correct_outputs)
# print(jodatime_dataset)

jodatime_test_examples, jodatime_y_true = get_test_examples_and_y_true(jodatime_dataset)
# print(jodatime_test_examples)

### All GWT Classes

In [ ]:
print("Total GWT Codes: {}\n".format(len(inputFiles[124:174])))
print(inputFiles[124:174])

gwt_codes = get_codes(inputFiles[124:174])
# print(gwt_codes)

gwt_correct_outputs = get_correct_outputs(correctOutputFiles[124:174])
# print(gwt_correct_outputs)

gwt_dataset = get_dataset(gwt_codes, gwt_correct_outputs)
# print(gwt_dataset)

gwt_test_examples, gwt_y_true = get_test_examples_and_y_true(gwt_dataset)
# print(gwt_test_examples)

### All XStream Classes

In [ ]:
print("Total XStream Codes: {}\n".format(len(inputFiles[224:268])))
print(inputFiles[224:268])

xstream_codes = get_codes(inputFiles[224:268])
# print(xstream_codes)

xstream_correct_outputs = get_correct_outputs(correctOutputFiles[224:268])
# print(xstream_correct_outputs)

xstream_dataset = get_dataset(xstream_codes, xstream_correct_outputs)
# print(xstream_dataset)

xstream_test_examples, xstream_y_true = get_test_examples_and_y_true(xstream_dataset)
# print(xstream_test_examples)

# One-shot Implementation

In [ ]:
MODEL = 'gpt-3.5-turbo'

In [ ]:
def pred_process(y_pred, y_true):
    y_pred_processed = []
    y_true_processed = []
    
    for pred, correct_imports in zip(y_pred, y_true):
        max_length = max(len(pred), len(correct_imports))
        correct_preds = list(set(pred).intersection(correct_imports))
#         print('Correct Predictions:', correct_preds)
#         wrong_preds = max_length - len(correct_preds)
#         print('Wrong Predictions:', wrong_preds)

        for i in range(0, max_length):
            if i<len(correct_preds):
                y_pred_processed.append(1)
                y_true_processed.append(1)
            else:
                if i<len(correct_imports):
                    y_pred_processed.append(0)
                    y_true_processed.append(1)
                else:
                    y_pred_processed.append(1)
                    y_true_processed.append(0)
            
    print(y_pred_processed)
    print(y_true_processed)
    print()
    return y_pred_processed, y_true_processed

In [ ]:
def eval_performance(y_pred, y_true):
    print(json.dumps({
        "accuracy": accuracy_score(y_pred, y_true),
        "f1": f1_score(y_pred, y_true),
        "precision": precision_score(y_pred, y_true),
        "recall": recall_score(y_pred, y_true)
    }, indent=2))

In [ ]:
# # Test

# print(xstream_test_examples[39][1]) # correct imports
# print(xstream_test_examples[39][0]) # code

In [ ]:
# # Test

# MODEL = 'gpt-3.5-turbo'

# code_snippet = xstream_test_examples[39][0]
# # prompt=f"Reply with to-the-point answer, with no elaboration. Extract all valid possible fully qualified type long-name from the code below, must exclude name of the class itself:\n\"\"\"{test_code}\"\"\""

# response = openai.ChatCompletion.create(
#     model=MODEL,
#     messages=[
#         {"role": "system", "content": "Reply with to-the-point answer, no elaboration."},
# #         {"role": "user", "content": f"Import correct imports:\n\"\"\"\n{test_code}\n\"\"\""},
# #         {"role": "user", "content": f"Your task is to provide a list of the correct imports for a given code snippet, ensuring that no wildcard imports are used. If any necessary imports are missing from the code, please include them in your response. Please note that your response should be specific and accurate:\n\"\"\"\n{code_snippet}\n\"\"\""},
# #         {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that your response should be specific and accurate:\n\"\"\"\n{code_snippet}\n\"\"\""},
#         {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay very close attention, and your response should be specific and accurate. You are prohibited from importing wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""},
        
#     ],
#     temperature=1,
# )

# print(response["choices"][0]["message"]["content"])

# One-shot with Random Example

In [ ]:
def get_oneshot_prediction(example, code):
    for code_snippet, correct_output in example:
        expected_output = '\n'.join(correct_output)
    
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Reply with to-the-point answer, no elaboration."},
            {"role": "system", "name":"example_user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""},
            {"role": "system", "name": "example_assistant", "content": f"{expected_output}"},
            {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code}\n\"\"\""},
        ],
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
def get_all_oneshot_predictions(example, dataset):
    y_pred = []
    for code_snippet, correct_imports in tqdm(dataset):
        if code_snippet in example[0][0]:
            predicted_output = get_oneshot_prediction([example[1]], code_snippet)
        else:
            predicted_output = get_oneshot_prediction([example[0]], code_snippet)
        y_pred.append(re.findall(r"import\s+[\w\., ]+;", predicted_output))
        time.sleep(5)
    return y_pred

In [ ]:
def get_oneshot_sample(dataset):
    oneshot_list = []
    
    for i in range(0, len(dataset["codes"])):
        oneshot_list.append(dict(codes=dataset["codes"][i], correct_outputs=dataset["correct_outputs"][i]))
        
    oneshot_example = [(example["codes"], example["correct_outputs"]) for example in oneshot_list]
    oneshot_sample = random.sample(oneshot_example, 2)
    
    # print(oneshot_sample)
    return oneshot_sample

In [ ]:
# One-shot Prediction for Android Classes

print("\nOne-shot Prediction for Android Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(android_dataset), android_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", android_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, android_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for JDK Classes

print("\nOne-shot Prediction for JDK Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(jdk_dataset), jdk_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jdk_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jdk_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for Hibernate Classes

print("\nOne-shot Prediction for Hibernate Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(hibernate_dataset), hibernate_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", hibernate_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, hibernate_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for Joda-Time Classes

print("\nOne-shot Prediction for Joda-Time Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(jodatime_dataset), jodatime_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jodatime_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jodatime_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for GWT Classes

print("\nOne-shot Prediction for GWT Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(gwt_dataset), gwt_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", gwt_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, gwt_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for XStream Classes

print("\nPrediction for XStream Classes:\n")
y_pred = get_all_oneshot_predictions(get_oneshot_sample(xstream_dataset), xstream_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", xstream_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, xstream_y_true)
eval_performance(y_pred_processed, y_true_processed)

# One-shot with Semantically Similar Example

In [ ]:
def get_semantic_oneshot_prediction(semantic_example, code):
    time.sleep(5)
    for code_snippet, example_output in semantic_example:
        expected_output = '\n'.join(example_output)
    
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Reply with to-the-point answer, no elaboration."},
            {"role": "system", "name":"example_user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""},
            {"role": "system", "name": "example_assistant", "content": f"{expected_output}"},
            {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code}\n\"\"\""},
        ],
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
def get_all_semantic_oneshot_predictions(semantic_example, dataset):
    y_pred = []
    i = 0
    for code, correct_imports in tqdm(dataset):
        predicted_output = get_semantic_oneshot_prediction([semantic_example[i]], code)
        y_pred.append(re.findall(r"import\s+[\w\., ]+;", predicted_output))
        i = i+1
    return y_pred

In [ ]:
def get_semantic_similarity(dataset):
    # Load pre-trained model and tokenizer
    model = RobertaModel.from_pretrained('microsoft/codebert-base')
    tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')

    # List to store code embeddings
    code_embeddings = []

    # Tokenize and encode all the code snippets and store the embeddings
    for code in dataset['codes']:
        inputs = tokenizer(code, return_tensors='pt', truncation=True, max_length=512, padding='max_length')
        with torch.no_grad():
            outputs = model(**inputs)
        embedding = outputs[0][0][0]
        code_embeddings.append(embedding)

    # Convert list to tensor for easier manipulation
    code_embeddings_tensor = torch.stack(code_embeddings)

    # Calculate pairwise cosine similarity
    cos_sim_matrix = torch.nn.functional.cosine_similarity(
        code_embeddings_tensor.unsqueeze(1), code_embeddings_tensor.unsqueeze(0), dim=2
    )

    # Fill the diagonal with large negative numbers to ignore self-comparison
    np.fill_diagonal(cos_sim_matrix.numpy(), float('-inf'))

    # Get the index of the most similar code for each code
    most_similar_indices = cos_sim_matrix.argmax(dim=1)

    # Construct a new list of tuples with similar codes and their own imports
    semantic_example_list = []
    for i, idx in enumerate(most_similar_indices):
        semantic_example_list.append((dataset['codes'][idx.item()], dataset['correct_outputs'][idx.item()]))

    return semantic_example_list

In [ ]:
# One-shot Prediction for Android Classes using semantic similarity

print("\nOne-shot Prediction for Android Classes using semantic similarity:\n")
android_semantic_list = get_semantic_similarity(android_dataset)
print(f"Android Semantic List:\n\n{android_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(android_semantic_list, android_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", android_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, android_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for JDK Classes using semantic similarity

print("\nOne-shot Prediction for JDK Classes using semantic similarity:\n")
jdk_semantic_list = get_semantic_similarity(jdk_dataset)
print(f"JDK Semantic List:\n\n{jdk_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(jdk_semantic_list, jdk_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jdk_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jdk_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for Hibernate Classes using semantic similarity

print("\nOne-shot Prediction for Hibernate Classes using semantic similarity:\n")
hibernate_semantic_list = get_semantic_similarity(hibernate_dataset)
print(f"Hibernate Semantic List:\n\n{hibernate_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(hibernate_semantic_list, hibernate_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", hibernate_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, hibernate_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for Joda-Time Classes using semantic similarity

print("\nOne-shot Prediction for Joda-Time Classes using semantic similarity:\n")
jodatime_semantic_list = get_semantic_similarity(jodatime_dataset)
print(f"Joda-Time Semantic List:\n\n{jodatime_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(jodatime_semantic_list, jodatime_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jodatime_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jodatime_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for GWT Classes using semantic similarity

print("\nOne-shot Prediction for GWT Classes using semantic similarity:\n")
gwt_semantic_list = get_semantic_similarity(gwt_dataset)
print(f"GWT Semantic List:\n\n{gwt_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(gwt_semantic_list, gwt_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", gwt_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, gwt_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# One-shot Prediction for XStream Classes using semantic similarity

print("\nOne-shot Prediction for XStream Classes using semantic similarity:\n")
xstream_semantic_list = get_semantic_similarity(xstream_dataset)
print(f"XStream Semantic List:\n\n{xstream_semantic_list}\n\n")
y_pred = get_all_semantic_oneshot_predictions(xstream_semantic_list, xstream_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", xstream_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, xstream_y_true)
eval_performance(y_pred_processed, y_true_processed)

# Appending Predicted Imports to the Code Body

In [ ]:
def append_imports(save_directory, code_name, list_of_java_codes, list_of_imports):
    code_import_dict = {}
    # Check if lengths match
    if len(list_of_java_codes) != len(list_of_imports):
        print("Mismatch between number of Java code strings and import lists!")
    else:
        # Zip the lists together into a dictionary
        code_import_dict = {i: (imports, code) for i, (imports, code) in enumerate(zip(list_of_imports, list_of_java_codes))}

    # Prepend the imports to the Java codes, save each to a .java file
    for index, (imports, code) in code_import_dict.items():
        # Split the code into lines
        lines = code.split('\n')
        
        # Find the line with the package declaration
        package_line_index = next((i for i, line in enumerate(lines) if line.strip().startswith('package ')), None)

        # If a package declaration is found, insert the imports after it
        if package_line_index is not None:
            lines = lines[:package_line_index+1] + imports + lines[package_line_index+1:]
        else:
            # If not, prepend the imports to the code
            lines = imports + lines

        full_code = '\n'.join(lines)
        file_name = ""
        if code_name == "android":
            if index<9:
                file_name = f"Android0{index+1}.java"
            else:
                file_name = f"Android{index+1}.java"
        elif code_name == "jdk":
            file_name = f"Class_{index+1}.java" 
        elif code_name == "hibernate":
            if index == 0:
                file_name = f"HibernateUtil.java"
            else:
                file_name = f"hibernate_class_{index}.java"
        elif code_name == "jodatime":
            if index<9:
                file_name = f"JodaTime0{index+1}.java"
            else:
                file_name = f"JodaTime{index+1}.java"
        elif code_name == "gwt":
            file_name = f"gwt_class_{index+1}.java"
        elif code_name == "xstream":
            file_name = f"xstream_class_{index+1}.java"
        full_path = os.path.join(save_directory, file_name)

        # Save the full code to a .java file
        with open(full_path, "w") as f:
            f.write(full_code)

In [ ]:
save_directory = "/home/azmain/Code for Compilation Test/one-shot/android"
code_name = "android"
codes = android_codes
predicted_imports = [['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;'], ['import android.app.Activity;', 'import android.os.Bundle;'], ['import android.graphics.drawable.Drawable;', 'import com.google.android.maps.ItemizedOverlay;', 'import com.google.android.maps.OverlayItem;', 'import com.google.android.maps.GeoPoint;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;'], ['import android.graphics.drawable.Drawable;', 'import android.widget.TabHost;', 'import android.widget.TabSpec;'], ['import android.view.Gravity;', 'import android.view.ViewGroup;', 'import android.widget.LinearLayout;', 'import android.view.View;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.view.View;', 'import android.widget.ArrayAdapter;'], ['import java.net.InetAddress;', 'import java.net.Inet4Address;', 'import java.net.UnknownHostException;', 'import java.io.IOException;', 'import java.net.Socket;', 'import android.util.Log;'], ['import android.os.Bundle;', 'import android.widget.TextView;', 'import java.util.Timer;', 'import java.util.TimerTask;'], ['import android.os.SystemClock;', 'import android.util.Log;', 'import android.widget.Chronometer;'], ['import android.content.Context;', 'import android.graphics.Canvas;', 'import android.location.Location;', 'import android.view.MotionEvent;', 'import com.google.android.maps.GeoPoint;', 'import com.google.android.maps.MapView;', 'import com.google.android.maps.MyLocationOverlay;'], ['import android.app.Activity;', 'import android.graphics.Color;', 'import android.os.Bundle;', 'import android.view.ViewGroup;', 'import android.widget.LinearLayout;', 'import android.widget.TextView;'], ['import android.content.ContentResolver;', 'import android.graphics.Bitmap;', 'import android.graphics.BitmapFactory;', 'import android.net.Uri;', 'import java.io.ByteArrayInputStream;', 'import java.io.ByteArrayOutputStream;', 'import java.io.FileNotFoundException;', 'import java.io.IOException;'], ['import java.io.File;', 'import java.io.FileNotFoundException;', 'import java.io.FileOutputStream;', 'import java.io.IOException;', 'import java.io.InputStream;', 'import java.net.HttpURLConnection;', 'import java.net.MalformedURLException;', 'import java.net.ProtocolException;', 'import java.net.URL;'], ['import android.content.Intent;', 'import android.net.Uri;'], ['import android.content.Context;', 'import android.graphics.Canvas;', 'import android.util.AttributeSet;', 'import android.util.Log;', 'import android.view.MotionEvent;', 'import android.view.View;', 'import android.widget.SeekBar;'], ['import java.io.IOException;', 'import java.io.InputStreamReader;', 'import java.net.URI;', 'import java.net.URISyntaxException;', 'import org.apache.http.HttpEntity;', 'import org.apache.http.HttpResponse;', 'import org.apache.http.client.HttpClient;', 'import org.apache.http.client.methods.HttpGet;', 'import org.apache.http.impl.client.DefaultHttpClient;', 'import android.util.Log;'], ['import android.app.Activity;', 'import android.content.DialogInterface;', 'import android.content.Intent;', 'import android.os.Bundle;', 'import android.view.View.OnClickListener;'], ['import android.content.Context;', 'import android.view.View;', 'import android.view.ViewGroup;', 'import android.widget.GridView;', 'import android.widget.ImageView;'], ['import android.content.Context;', 'import android.view.View;', 'import android.view.animation.ScaleAnimation;', 'import android.widget.TextView;'], ['import android.content.Context;', 'import android.widget.ImageButton;', 'import android.widget.LinearLayout;'], ['import android.app.ListActivity;', 'import android.content.Intent;', 'import android.database.Cursor;', 'import android.os.Bundle;', 'import android.provider.Contacts.People;', 'import android.view.View;', 'import android.widget.ListView;', 'import android.widget.SimpleCursorAdapter;', 'import android.widget.TextView;'], ['import android.app.Activity;', 'import android.graphics.Bitmap;', 'import android.graphics.BitmapFactory;', 'import android.os.Bundle;', 'import android.widget.ArrayAdapter;', 'import android.widget.ImageView;', 'import android.widget.ListView;', 'import java.io.IOException;', 'import java.net.HttpURLConnection;', 'import java.net.MalformedURLException;', 'import java.net.URL;', 'import java.util.ArrayList;', 'import java.util.Random;'], ['import android.app.Activity;', 'import android.content.Context;', 'import android.location.LocationManager;', 'import android.os.Bundle;', 'import android.util.Log;', 'import java.util.Iterator;', 'import java.util.List;'], ['import android.app.ListActivity;', 'import android.content.Context;', 'import android.database.sqlite.SQLiteDatabase;', 'import android.os.Bundle;', 'import java.util.ArrayList;'], ['import android.content.Context;', 'import android.view.KeyEvent;', 'import android.view.MotionEvent;', 'import android.view.View;', 'import android.view.View.OnTouchListener;', 'import android.widget.ScrollView;'], ['import android.content.Context;', 'import android.view.View;', 'import android.view.ViewGroup;', 'import android.widget.BaseAdapter;', 'import android.widget.GridView;', 'import android.widget.TextView;'], ['import android.app.Application;', 'import android.content.Context;'], ['import android.app.Service;', 'import android.content.Context;', 'import android.content.Intent;', 'import android.os.IBinder;', 'import android.os.PowerManager;'], ['import java.io.IOException;', 'import org.apache.http.HttpEntity;', 'import org.apache.http.HttpHost;', 'import org.apache.http.HttpResponse;', 'import org.apache.http.client.HttpClient;', 'import org.apache.http.client.methods.HttpGet;', 'import org.apache.http.impl.client.DefaultHttpClient;', 'import org.apache.http.util.EntityUtils;'], ['import android.content.Context;', 'import android.os.PowerManager;'], ['import android.os.Environment;', 'import android.media.MediaRecorder;', 'import java.io.File;', 'import java.io.IOException;'], ['import android.content.Context;', 'import android.graphics.PixelFormat;', 'import android.hardware.Camera;', 'import android.hardware.Camera.Parameters;', 'import android.view.SurfaceHolder;', 'import java.io.IOException;'], ['import android.app.TabActivity;', 'import android.os.Bundle;', 'import android.util.Log;', 'import android.view.View;', 'import android.view.View.OnClickListener;', 'import android.widget.TabHost;', 'import android.widget.TabHost.OnTabChangeListener;', 'import android.widget.TabHost.TabContentFactory;'], ['import android.media.MediaRecorder;', 'import java.io.File;', 'import java.io.IOException;'], ['import android.content.ContentValues;', 'import android.database.sqlite.SQLiteDatabase;'], ['import android.app.Activity;', 'import android.app.ProgressDialog;', 'import android.os.Handler;', 'import android.os.Message;'], ['import java.io.DataInputStream;', 'import java.io.DataOutputStream;', 'import java.io.File;', 'import java.io.FileInputStream;', 'import java.io.IOException;', 'import java.io.InputStream;', 'import java.net.HttpURLConnection;', 'import java.net.MalformedURLException;', 'import java.net.URL;'], ['import androidExamples.Android39;', 'import android.os.Bundle;', 'import android.view.View;', 'import android.widget.List;', 'import com.google.android.maps.MapActivity;', 'import com.google.android.maps.MapController;', 'import com.google.android.maps.MapView;', 'import com.google.android.maps.MyLocationOverlay;', 'import com.google.android.maps.Overlay;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.util.Log;', 'import android.view.View;', 'import android.view.View.OnClickListener;', 'import android.widget.ImageView;', 'import android.widget.SlidingDrawer;'], ['import android.content.Context;', 'import android.graphics.Color;', 'import android.view.View;', 'import android.widget.TextView;'], ['import android.app.Activity;', 'import android.content.Intent;', 'import android.content.pm.ActivityInfo;', 'import android.graphics.PixelFormat;', 'import android.media.MediaPlayer;', 'import android.media.MediaPlayer.OnBufferingUpdateListener;', 'import android.media.MediaPlayer.OnCompletionListener;', 'import android.media.MediaPlayer.OnErrorListener;', 'import android.media.MediaPlayer.OnPreparedListener;', 'import android.net.Uri;', 'import android.os.Bundle;', 'import android.util.Log;', 'import android.view.SurfaceHolder;', 'import android.view.SurfaceView;', 'import android.view.View;', 'import android.webkit.URLUtil;', 'import android.widget.Button;', 'import java.io.File;', 'import java.io.FileOutputStream;', 'import java.io.IOException;', 'import java.io.InputStream;', 'import java.net.URL;', 'import java.net.URLConnection;'], ['import java.io.IOException;', 'import org.apache.http.HttpEntity;', 'import org.apache.http.HttpResponse;', 'import org.apache.http.client.HttpClient;', 'import org.apache.http.client.methods.HttpGet;', 'import org.apache.http.impl.client.DefaultHttpClient;', 'import org.apache.http.params.HttpParams;', 'import org.apache.http.util.EntityUtils;', 'import org.apache.http.HttpHost;'], ['import android.app.Service;', 'import android.content.Intent;', 'import android.os.IBinder;'], ['import android.app.Activity;', 'import android.content.BroadcastReceiver;', 'import android.content.ComponentName;', 'import android.content.Context;', 'import android.content.Intent;', 'import android.content.IntentFilter;', 'import android.os.Bundle;', 'import android.util.Log;', 'import android.webkit.MalformedMimeTypeException;'], ['import android.app.Activity;', 'import android.content.Intent;', 'import android.os.Bundle;', 'import android.view.View;', 'import android.widget.Button;', 'import android.widget.EditText;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;', 'import org.apache.http.client.ClientProtocolException;', 'import org.apache.http.client.methods.HttpGet;', 'import org.apache.http.impl.client.DefaultHttpClient;', 'import java.io.IOException;'], ['import android.app.AlertDialog;', 'import android.app.Dialog;', 'import android.content.Context;', 'import android.graphics.Bitmap;', 'import android.graphics.BitmapFactory;', 'import android.hardware.Camera;', 'import android.hardware.Camera.PictureCallback;', 'import android.widget.ImageView;', 'import android.widget.TextView;'], ['import android.content.Context;', 'import android.util.AttributeSet;', 'import android.view.Gravity;', 'import android.view.LayoutInflater;', 'import android.widget.LinearLayout;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.util.Log;', 'import android.view.View;', 'import android.widget.Button;', 'import android.widget.EditText;', 'import android.widget.TextView;', 'import java.text.NumberFormat;']]
import_list = predicted_imports
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
save_directory = "/home/azmain/Code for Compilation Test/one-shot/jdk"
code_name = "jdk"
codes = jdk_codes
predicted_imports = [['import java.applet.Applet;', 'import java.awt.Color;', 'import java.awt.Container;', 'import java.awt.Dimension;', 'import java.awt.Frame;', 'import java.awt.Graphics;', 'import java.awt.Graphics2D;', 'import java.awt.event.WindowAdapter;', 'import java.awt.event.WindowEvent;', 'import java.awt.image.BufferedImage;', 'import java.io.FileInputStream;', 'import java.io.FileOutputStream;', 'import java.io.IOException;', 'import java.io.ObjectInputStream;', 'import java.io.ObjectOutputStream;'], ['import java.awt.BorderLayout;', 'import java.awt.Color;', 'import java.awt.Dimension;', 'import java.awt.FlowLayout;', 'import java.awt.GradientPaint;', 'import java.awt.Graphics2D;', 'import java.awt.GridBagLayout;', 'import java.awt.GridLayout;', 'import java.awt.event.ActionEvent;', 'import java.awt.event.ActionListener;', 'import java.awt.image.BufferedImage;', 'import javax.swing.JButton;', 'import javax.swing.JCheckBox;', 'import javax.swing.JComboBox;', 'import javax.swing.JFrame;', 'import javax.swing.JLabel;', 'import javax.swing.JPanel;', 'import javax.swing.JScrollPane;', 'import javax.swing.JSplitPane;', 'import javax.swing.JTable;', 'import javax.swing.SwingUtilities;', 'import javax.swing.UIManager;', 'import javax.swing.border.TitledBorder;', 'import javax.swing.table.DefaultTableModel;'], ['import java.math.RoundingMode;', 'import java.text.DecimalFormat;', 'import java.text.DecimalFormatSymbols;', 'import java.text.ParseException;', 'import java.util.Locale;'], ['import java.util.ArrayList;', 'import java.util.List;', 'import java.util.regex.Matcher;', 'import java.util.regex.Pattern;'], ['import java.util.Map;', 'import java.util.HashMap;', 'import java.util.List;', 'import java.util.LinkedList;', 'import java.util.Collections;', 'import java.util.Comparator;', 'import java.util.LinkedHashMap;', 'import java.util.TreeMap;'], ['import java.awt.BorderLayout;', 'import java.awt.Color;', 'import java.awt.Component;', 'import java.awt.Graphics;', 'import java.awt.Graphics2D;', 'import java.awt.RenderingHints;', 'import java.awt.event.ActionEvent;', 'import java.awt.event.ActionListener;', 'import java.util.Observable;', 'import java.util.Observer;', 'import java.util.Random;', 'import javax.swing.Icon;', 'import javax.swing.JButton;', 'import javax.swing.JFrame;', 'import javax.swing.JLabel;', 'import javax.swing.JPanel;', 'import java.awt.EventQueue;'], ['import java.text.ParseException;', 'import java.text.SimpleDateFormat;', 'import java.util.ArrayList;', 'import java.util.Collections;', 'import java.util.Comparator;', 'import java.util.Date;'], ['import java.lang.Math;'], ['import java.sql.Connection;', 'import java.sql.DriverManager;', 'import java.sql.SQLException;', 'import java.util.Properties;'], ['import java.text.NumberFormat;', 'import java.util.Locale;'], ['import java.io.BufferedWriter;', 'import java.io.File;', 'import java.io.FileWriter;', 'import javax.swing.JFileChooser;'], ['import java.text.DateFormat;', 'import java.text.ParseException;', 'import java.text.SimpleDateFormat;', 'import java.util.Calendar;', 'import java.util.Date;', 'import java.util.GregorianCalendar;'], ['import javax.swing.JFrame;', 'import javax.swing.JTextArea;', 'import javax.swing.SwingUtilities;', 'import javax.swing.UIManager;'], ['import java.lang.reflect.Method;'], ['import java.util.HashMap;'], ['import java.lang.reflect.InvocationTargetException;', 'import java.lang.reflect.Method;', 'import java.util.ArrayList;', 'import java.util.HashMap;', 'import java.util.List;', 'import java.util.Map;', 'import java.util.prefs.Preferences;'], ['import java.text.ParseException;', 'import java.text.SimpleDateFormat;', 'import java.time.format.DateTimeFormatter;', 'import java.time.temporal.TemporalAccessor;', 'import java.util.Date;'], ['import java.lang.reflect.Array;', 'import java.lang.reflect.Field;'], ['import java.util.Random;', 'import java.util.concurrent.ThreadLocalRandom;', 'import java.util.List;', 'import java.util.ArrayList;', 'import java.util.Collection;', 'import java.util.Iterator;', 'import java.util.NoSuchElementException;'], ['import java.io.DataInputStream;', 'import java.io.DataOutputStream;', 'import java.io.BufferedInputStream;', 'import java.io.BufferedOutputStream;', 'import java.io.File;', 'import java.io.FileOutputStream;', 'import java.io.FileInputStream;', 'import java.io.IOException;', 'import java.util.ArrayList;'], ['import java.awt.BorderLayout;', 'import java.awt.Component;', 'import java.awt.Image;', 'import java.awt.event.ActionEvent;', 'import java.awt.event.ActionListener;', 'import java.awt.event.InputEvent;', 'import java.awt.event.KeyEvent;', 'import java.awt.image.BufferedImage;', 'import java.io.File;', 'import javax.imageio.ImageIO;', 'import javax.swing.ImageIcon;', 'import javax.swing.JFrame;', 'import javax.swing.JLabel;', 'import javax.swing.JMenu;', 'import javax.swing.JMenuBar;', 'import javax.swing.JMenuItem;', 'import javax.swing.JOptionPane;', 'import javax.swing.JPanel;', 'import javax.swing.JScrollPane;', 'import javax.swing.JTextArea;', 'import javax.swing.JTree;', 'import javax.swing.KeyStroke;', 'import javax.swing.SwingUtilities;', 'import javax.swing.border.TitledBorder;'], ['import java.awt.Container;', 'import java.awt.EventQueue;', 'import java.awt.GridLayout;', 'import java.beans.PropertyChangeEvent;', 'import java.beans.PropertyChangeListener;', 'import java.text.DecimalFormat;', 'import java.util.List;', 'import javax.swing.JFrame;', 'import javax.swing.JLabel;', 'import javax.swing.JProgressBar;', 'import javax.swing.SwingWorker;'], []]
import_list = predicted_imports
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
save_directory = "/home/azmain/Code for Compilation Test/one-shot/hibernate"
code_name = "hibernate"
codes = hibernate_codes
predicted_imports = [['import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.AnnotationConfiguration;'], ['import org.hibernate.SessionFactory;', 'import org.hibernate.Session;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.AnnotationConfiguration;', 'import hibernate.User;', 'import hibernate.UserDAO;'], ['import java.io.Serializable;', 'import java.util.List;', 'import javax.persistence.Cache;', 'import javax.persistence.CacheConcurrencyStrategy;', 'import javax.persistence.Column;', 'import javax.persistence.Entity;', 'import javax.persistence.Id;', 'import javax.persistence.JoinColumn;', 'import javax.persistence.ManyToOne;', 'import javax.persistence.OneToMany;'], ['import java.sql.Types;', 'import org.hibernate.dialect.Dialect;', 'import org.hibernate.dialect.function.SQLFunctionTemplate;', 'import org.hibernate.dialect.function.StandardSQLFunction;', 'import org.hibernate.dialect.function.VarArgsSQLFunction;', 'import org.hibernate.type.Hibernate;'], ['import org.hibernate.HibernateException;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.Configuration;'], ['import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.Configuration;'], ['import java.io.File;', 'import java.net.MalformedURLException;', 'import java.util.Iterator;', 'import java.util.List;', 'import org.dom4j.Document;', 'import org.dom4j.DocumentException;', 'import org.dom4j.io.SAXReader;', 'import org.hibernate.HibernateException;', 'import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.Configuration;', 'import org.hibernate.cfg.SchemaExport;', 'import org.hibernate.engine.spi.EntityMode;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.AnnotationConfiguration;'], ['import java.util.Date;', 'import javax.persistence.Column;', 'import javax.persistence.MappedSuperclass;', 'import javax.persistence.Temporal;', 'import javax.persistence.TemporalType;', 'import javax.validation.constraints.AssertTrue;', 'import javax.validation.constraints.NotNull;'], ['import java.io.Serializable;', 'import java.util.List;', 'import org.hibernate.Session;'], ['import java.io.Serializable;', 'import java.util.List;', 'import org.hibernate.Session;'], ['import javax.persistence.Entity;', 'import javax.persistence.Table;', 'import javax.persistence.Id;', 'import javax.persistence.Column;', 'import javax.persistence.ManyToOne;', 'import javax.persistence.JoinColumn;', 'import javax.persistence.ForeignKey;', 'import javax.persistence.FetchType;', 'import java.io.Serializable;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.Configuration;', 'import javax.faces.context.FacesContext;'], ['import java.io.Serializable;', 'import java.util.Collection;', 'import java.util.List;', 'import javax.persistence.Column;', 'import javax.persistence.Entity;', 'import javax.persistence.FetchType;', 'import javax.persistence.ForeignKey;', 'import javax.persistence.GeneratedValue;', 'import javax.persistence.Id;', 'import javax.persistence.JoinColumn;', 'import javax.persistence.JoinTable;', 'import javax.persistence.ManyToMany;', 'import javax.persistence.ManyToOne;', 'import javax.persistence.OneToMany;', 'import javax.persistence.Table;', 'import org.hibernate.annotations.IndexColumn;', 'import library.model.County;', 'import library.model.Permission;', 'import library.model.Version;'], ['import java.io.Serializable;', 'import java.util.Collection;', 'import javax.persistence.Column;', 'import javax.persistence.Entity;', 'import javax.persistence.FetchType;', 'import javax.persistence.ForeignKey;', 'import javax.persistence.Id;', 'import javax.persistence.JoinColumn;', 'import javax.persistence.ManyToMany;', 'import javax.persistence.ManyToOne;', 'import javax.persistence.Table;', 'import org.hibernate.annotations.Cache;', 'import org.hibernate.annotations.CacheConcurrencyStrategy;', 'import org.hibernate.annotations.CascadeType;'], ['import java.io.ByteArrayOutputStream;', 'import java.io.IOException;', 'import java.io.InputStream;', 'import java.sql.Blob;', 'import java.sql.SQLException;', 'import java.util.Date;', 'import org.hibernate.Hibernate;'], ['import java.util.List;', 'import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import hibernate.Event;'], ['import java.util.ArrayList;', 'import java.util.List;', 'import javax.naming.NamingException;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.Configuration;', 'import org.hibernate.shards.ShardConfiguration;', 'import org.hibernate.shards.ShardId;', 'import org.hibernate.shards.ShardResolutionStrategy;', 'import org.hibernate.shards.ShardSelectionStrategy;', 'import org.hibernate.shards.ShardStrategy;', 'import org.hibernate.shards.ShardStrategyFactory;', 'import org.hibernate.shards.access.SequentialShardAccessStrategy;', 'import org.hibernate.shards.loadbalance.RoundRobinShardLoadBalancer;', 'import org.hibernate.shards.strategy.AllShardsShardResolutionStrategy;', 'import org.hibernate.shards.strategy.RoundRobinShardSelectionStrategy;', 'import org.hibernate.shards.strategy.ShardedConfiguration;', 'import org.hibernate.shards.strategy.ShardStrategyImpl;', 'import org.hibernate.shards.util.ConfigurationToShardConfigurationAdapter;'], ['import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.Configuration;'], ['import javax.persistence.Column;', 'import javax.persistence.Entity;', 'import javax.persistence.GeneratedValue;', 'import javax.persistence.Id;', 'import org.hibernate.annotations.Index;', 'import org.hibernate.annotations.Table;'], ['import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.AnnotationConfiguration;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.Configuration;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.Configuration;', 'import org.hibernate.query.Query;', 'import com.google.gwt.json.client.JSONObject;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.AnnotationConfiguration;', 'import org.hibernate.HibernateException;', 'import org.hibernate.cfg.Configuration;'], ['import java.lang.reflect.InvocationHandler;', 'import java.lang.reflect.Method;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.Configuration;'], ['import javax.persistence.Entity;', 'import javax.persistence.GeneratedValue;', 'import javax.persistence.Id;', 'import javax.persistence.OneToOne;', 'import org.hibernate.annotations.CascadeType;', 'import org.hibernate.annotations.Fetch;', 'import org.hibernate.annotations.FetchMode;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.AnnotationConfiguration;', 'import org.hibernate.tool.hbm2ddl.SchemaExport;', 'import javax.persistence.Entity;', 'import javax.persistence.GeneratedValue;', 'import javax.persistence.Id;', 'import javax.persistence.OneToOne;', 'import javax.persistence.CascadeType;', 'import org.hibernate.annotations.Fetch;', 'import org.hibernate.annotations.FetchMode;', 'import org.hibernate.query.Query;', 'import java.util.List;'], ['import java.util.List;', 'import org.hibernate.Session;', 'import org.hibernate.Query;'], ['import java.util.ArrayList;', 'import java.util.List;', 'import javax.persistence.Column;', 'import javax.persistence.Entity;', 'import javax.persistence.GeneratedValue;', 'import javax.persistence.Id;', 'import javax.persistence.JoinColumn;', 'import javax.persistence.ManyToOne;', 'import javax.persistence.OneToMany;', 'import org.hibernate.annotations.Index;', 'import org.hibernate.annotations.IndexColumn;', 'import java.io.Serializable;', 'import javax.persistence.GenerationType;'], ['import org.hibernate.Criteria;', 'import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.criterion.Projections;'], ['import java.io.Serializable;', 'import javax.persistence.Basic;', 'import javax.persistence.Column;', 'import javax.persistence.Entity;', 'import javax.persistence.Id;', 'import javax.persistence.NamedQueries;', 'import javax.persistence.NamedQuery;', 'import javax.persistence.Table;'], ['import java.util.List;', 'import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.Configuration;', 'import org.hibernate.query.Query;'], ['import javax.persistence.Entity;', 'import javax.persistence.GeneratedValue;', 'import javax.persistence.Id;', 'import javax.persistence.Table;', 'import javax.persistence.Index;', 'import org.hibernate.annotations.Table;', 'import javax.persistence.Column;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.Configuration;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;'], ['import org.apache.log4j.BasicConfigurator;', 'import org.hibernate.HibernateException;', 'import org.hibernate.Query;', 'import org.hibernate.Session;', 'import org.hibernate.Transaction;', 'import java.util.Iterator;', 'import java.util.List;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.Configuration;'], ['import org.hibernate.HibernateException;', 'import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.AnnotationConfiguration;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.Configuration;'], ['import java.io.Serializable;', 'import java.util.List;', 'import java.util.Map;', 'import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;', 'import org.springframework.transaction.annotation.Transactional;', 'import org.hibernate.search.SearchException;'], ['import java.util.Iterator;', 'import java.util.List;', 'import org.hibernate.Query;', 'import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.Configuration;'], ['import org.hibernate.Criteria;', 'import org.hibernate.HibernateException;', 'import org.hibernate.Session;', 'import org.hibernate.Transaction;', 'import org.hibernate.criterion.Restrictions;', 'import java.util.List;'], ['import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.Configuration;', 'import org.apache.commons.logging.Log;', 'import org.apache.commons.logging.LogFactory;', 'import javax.naming.InitialContext;', 'import java.util.List;', 'import org.hibernate.criterion.Example;'], ['import org.apache.log4j.Logger;', 'import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.cfg.Configuration;', 'import java.util.Properties;'], ['import java.util.ArrayList;', 'import java.util.HashSet;', 'import java.util.Iterator;', 'import java.util.Set;', 'import javax.servlet.http.HttpServletRequest;', 'import javax.servlet.http.HttpServletResponse;', 'import javax.servlet.http.HttpSession;', 'import org.apache.struts.action.ActionForm;', 'import org.apache.struts.action.ActionForward;', 'import org.apache.struts.action.ActionMapping;', 'import org.apache.struts.actions.DispatchAction;', 'import org.hibernate.Query;', 'import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.AnnotationConfiguration;'], ['import org.hibernate.HibernateException;', 'import org.hibernate.JDBCException;', 'import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.CacheMode;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.Configuration;', 'import com.example.hibernate.HibernateUtil;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import org.hibernate.Transaction;', 'import org.hibernate.cfg.Configuration;', 'import com.google.gwt.user.server.rpc.RemoteServiceServlet;', 'import com.example.client.GreetingService;', 'import com.example.server.HibernateUtil;'], ['import java.io.Serializable;', 'import javax.persistence.Entity;', 'import javax.persistence.Version;', 'import org.hibernate.annotations.NaturalId;'], ['import org.hibernate.Session;', 'import org.hibernate.SessionFactory;', 'import java.util.List;']]
import_list = predicted_imports
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
save_directory = "/home/azmain/Code for Compilation Test/one-shot/jodatime"
code_name = "jodatime"
codes = jodatime_codes
predicted_imports = [['import org.joda.time.DateTime;', 'import org.joda.time.DateTimeZone;', 'import org.joda.time.format.DateTimeFormatter;', 'import org.joda.time.format.ISODateTimeFormat;', 'import java.util.TimeZone;'], ['import org.joda.time.DateMidnight;', 'import org.joda.time.DateTimeZone;', 'import org.joda.time.DateTime;'], ['import org.joda.time.DateTime;', 'import org.joda.time.Interval;', 'import org.joda.time.format.PeriodFormatter;', 'import org.joda.time.format.PeriodFormatterBuilder;'], ['import org.joda.time.Period;', 'import org.joda.time.ReadableInstant;'], ['import org.joda.time.DateTime;', 'import org.joda.time.DateTimeZone;', 'import org.joda.time.chrono.GJChronology;', 'import org.joda.time.Chronology;'], ['import org.joda.time.Duration;', 'import org.joda.time.PeriodType;', 'import org.joda.time.format.PeriodFormatterBuilder;'], ['import java.text.DateFormat;', 'import java.text.ParseException;', 'import java.text.SimpleDateFormat;', 'import java.util.Calendar;', 'import java.util.Date;', 'import org.joda.time.DateTime;', 'import org.joda.time.Days;'], ['import org.joda.time.Period;', 'import org.joda.time.PeriodType;', 'import java.util.concurrent.TimeUnit;'], ['import org.joda.time.DateTimeZone;', 'import java.util.TimeZone;'], ['import java.io.IOException;', 'import java.io.ObjectInputStream;', 'import java.io.ObjectOutputStream;', 'import java.io.Serializable;', 'import org.joda.time.DateTimeZone;', 'import org.joda.time.chrono.ISOChronology;'], ['import org.joda.time.DateTime;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;', 'import java.text.ParseException;', 'import java.text.SimpleDateFormat;'], ['import java.text.ParseException;', 'import java.text.SimpleDateFormat;', 'import java.util.Date;', 'import org.joda.time.DateTime;'], ['import org.junit.Test;', 'import static org.junit.Assert.assertEquals;', 'import org.joda.time.Period;', 'import org.joda.time.format.PeriodFormatter;', 'import org.joda.time.format.PeriodFormatterBuilder;'], ['import org.joda.time.DateTime;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;'], ['import org.joda.time.MutableDateTime;', 'import org.joda.time.DateTime;'], ['import org.joda.time.DateTime;', 'import org.joda.time.Period;', 'import org.joda.time.format.PeriodFormatter;', 'import org.joda.time.format.PeriodFormatterBuilder;'], ['import org.joda.time.DateTime;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;'], ['import org.joda.time.DateTimeZone;', 'import org.joda.time.format.DateTimeFormatter;', 'import org.joda.time.format.DateTimeFormatterBuilder;'], ['import org.joda.time.Instant;', 'import org.joda.time.Interval;', 'import org.joda.time.ReadableInstant;', 'import java.util.Date;'], ['import org.joda.time.DateTime;', 'import org.joda.time.DateTimeZone;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;'], ['import java.text.DateFormat;', 'import java.text.ParseException;', 'import java.text.SimpleDateFormat;', 'import java.util.Date;', 'import org.joda.time.DateTime;', 'import org.joda.time.DateTimeZone;', 'import org.joda.time.LocalDateTime;'], ['import org.joda.time.DateTime;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;', 'import java.text.ParseException;', 'import java.util.Date;'], ['import org.joda.time.LocalDate;', 'import org.joda.time.DateTimeConstants;'], ['import org.joda.time.DateTime;', 'import org.joda.time.Period;', 'import org.joda.time.PeriodFormat;', 'import org.joda.time.PeriodType;'], ['import org.joda.time.DateTime;', 'import org.joda.time.Days;', 'import org.joda.time.DateMidnight;', 'import org.joda.time.LocalDateTime;'], ['import org.joda.time.Duration;', 'import org.joda.time.Period;'], ['import org.joda.time.DateTime;', 'import java.util.ArrayList;'], ['import org.joda.time.DateTime;', 'import org.joda.time.Days;', 'import org.joda.time.LocalDate;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;', 'import java.util.ArrayList;', 'import java.util.List;'], ['import org.joda.time.DateTime;', 'import org.joda.time.format.DateTimeFormatter;', 'import org.joda.time.format.IllegalFieldValueException;'], ['import org.joda.time.DateTime;', 'import org.joda.time.DateTimeZone;'], ['import org.joda.time.DateTime;', 'import org.joda.time.Weeks;'], ['import org.joda.time.DateTime;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;'], ['import org.joda.time.Period;', 'import org.joda.time.format.PeriodFormatter;', 'import org.joda.time.format.PeriodFormatterBuilder;'], ['import java.util.Locale;', 'import org.joda.time.Period;', 'import org.joda.time.format.PeriodFormat;', 'import org.joda.time.format.PeriodFormatter;'], ['import java.util.Date;', 'import org.joda.time.DateTime;', 'import org.joda.time.Period;', 'import org.joda.time.PeriodFormatterBuilder;', 'import org.joda.time.ReadablePeriod;'], ['import org.joda.time.Period;', 'import org.joda.time.format.ISOPeriodFormat;', 'import org.joda.time.format.PeriodFormatter;', 'import org.joda.time.format.PeriodFormatterBuilder;'], ['import org.joda.time.DateTime;', 'import org.joda.time.DateTimeZone;', 'import java.util.concurrent.TimeUnit;'], ['import org.joda.time.DateTime;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;'], ['import org.joda.time.DateTime;'], ['import org.joda.time.DateTime;', 'import org.joda.time.Days;'], ['import org.joda.time.DateTime;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;'], ['import org.joda.time.LocalDate;'], ['import org.joda.time.DateTime;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;', 'import org.joda.time.format.DateTimeFormatterBuilder;', 'import org.joda.time.format.DateTimeParser;'], ['import org.joda.time.Period;', 'import org.joda.time.format.PeriodFormatter;', 'import org.joda.time.format.PeriodFormatterBuilder;'], ['import java.util.Date;', 'import java.util.HashSet;', 'import java.util.Set;', 'import java.util.TimeZone;', 'import org.joda.time.DateTimeZone;', 'import org.joda.time.tz.Provider;'], ['import org.joda.time.DateTime;', 'import org.joda.time.DateTimeZone;'], ['import org.joda.time.DateTime;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;'], ['import org.joda.time.DateTime;', 'import org.joda.time.MillisProvider;'], ['import org.joda.time.DateTime;', 'import org.joda.time.LocalDate;', 'import org.joda.time.format.DateTimeFormat;', 'import org.joda.time.format.DateTimeFormatter;'], ['import org.joda.time.DateTime;']]
import_list = predicted_imports
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
save_directory = "/home/azmain/Code for Compilation Test/one-shot/gwt"
code_name = "gwt"
codes = gwt_codes
predicted_imports = [['import com.google.gwt.event.dom.client.MouseDownEvent;', 'import com.google.gwt.event.dom.client.MouseDownHandler;', 'import com.google.gwt.user.client.ui.AbsolutePanel;', 'import com.google.gwt.user.client.ui.Composite;'], ['import com.google.gwt.core.client.GWT;', 'import com.google.gwt.junit.client.GWTTestCase;', 'import com.google.gwt.user.client.rpc.AsyncCallback;', 'import com.google.gwt.sample.stockwatcher.client.GreetingService;', 'import com.google.gwt.sample.stockwatcher.client.GreetingServiceAsync;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.user.client.Window;', 'import com.google.gwt.user.client.rpc.AsyncCallback;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.TextBox;'], ['import com.google.gwt.user.client.ui.Composite;', 'import com.google.gwt.user.client.ui.HTML;', 'import com.google.gwt.user.client.ui.VerticalSplitPanel;'], ['import com.google.gwt.core.client.JsArray;', 'import com.google.gwt.core.client.JsArrayInteger;'], ['import com.google.gwt.user.client.DOM;', 'import com.google.gwt.user.client.Element;', 'import com.google.gwt.user.client.ui.Widget;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.core.client.GWT;', 'import com.google.gwt.user.client.Timer;', 'import com.google.gwt.user.client.ui.Panel;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.VerticalPanel;', 'import com.google.gwt.visualization.client.DataTable;', 'import com.google.gwt.visualization.client.visualizations.Table;', 'import com.google.gwt.visualization.client.visualizations.Table.Options;', 'import com.google.gwt.visualization.client.visualizations.Table.PACKAGE;'], ['import com.google.gwt.http.client.Request;', 'import com.google.gwt.http.client.RequestBuilder;', 'import com.google.gwt.http.client.RequestCallback;', 'import com.google.gwt.http.client.RequestException;', 'import com.google.gwt.http.client.Response;', 'import com.google.gwt.user.client.Window;'], ['import com.google.gwt.user.client.Command;', 'import com.google.gwt.user.client.ui.MenuBar;', 'import com.google.gwt.user.client.ui.VerticalPanel;'], ['import com.google.gwt.event.dom.client.ClickEvent;', 'import com.google.gwt.event.dom.client.ClickHandler;', 'import com.google.gwt.event.dom.client.ClickListener;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.FormPanel;', 'import com.google.gwt.user.client.ui.Label;', 'import com.google.gwt.user.client.ui.TextBox;', 'import com.google.gwt.user.client.ui.VerticalPanel;', 'import com.google.gwt.user.client.ui.Widget;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.user.client.ui.HorizontalPanel;', 'import com.google.gwt.user.client.ui.Label;', 'import com.google.gwt.user.client.ui.RootPanel;'], ['import com.google.gwt.user.client.ui.AbsolutePanel;', 'import com.google.gwt.user.client.ui.Label;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.TextBox;', 'import com.google.gwt.user.client.ui.VerticalPanel;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.user.client.ui.AbsolutePanel;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.ClickListener;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.Widget;'], ['import com.google.gwt.core.client.GWT;', 'import com.google.gwt.http.client.Request;', 'import com.google.gwt.http.client.RequestBuilder;', 'import com.google.gwt.http.client.RequestCallback;', 'import com.google.gwt.http.client.Response;', 'import com.google.gwt.user.client.Window;'], ['import com.google.gwt.http.client.Request;', 'import com.google.gwt.http.client.RequestBuilder;', 'import com.google.gwt.http.client.RequestCallback;', 'import com.google.gwt.http.client.RequestException;', 'import com.google.gwt.http.client.Response;', 'import com.google.gwt.http.client.URL;'], ['import com.google.gwt.http.client.Request;', 'import com.google.gwt.http.client.RequestBuilder;', 'import com.google.gwt.http.client.RequestCallback;', 'import com.google.gwt.http.client.Response;', 'import com.google.gwt.user.client.ui.HTML;', 'import com.google.gwt.user.client.ui.RootPanel;'], ['import com.google.gwt.user.client.ui.Composite;', 'import com.google.gwt.user.client.ui.Image;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.DockPanel;', 'import com.google.gwt.user.client.ui.HTML;', 'import com.google.gwt.user.client.ui.VerticalPanel;', 'import com.google.gwt.user.client.ui.AbstractImagePrototype;', 'import com.google.gwt.user.client.ui.ImageBundle;'], ['import com.google.gwt.core.client.GWT;', 'import com.google.gwt.uibinder.client.UiBinder;', 'import com.google.gwt.uibinder.client.UiBinder;', 'import com.google.gwt.user.client.ui.Composite;', 'import com.google.gwt.user.client.ui.Widget;'], ['import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.VerticalPanel;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.event.dom.client.ClickEvent;', 'import com.google.gwt.event.dom.client.ClickHandler;', 'import com.google.gwt.user.client.Window;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.RootPanel;'], ['import com.google.gwt.user.client.ui.PopupPanel;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.VerticalPanel;', 'import com.google.gwt.user.client.ui.Widget;'], ['import com.google.gwt.core.client.GWT;', 'import com.google.gwt.event.dom.client.ClickEvent;', 'import com.google.gwt.event.dom.client.ClickHandler;', 'import com.google.gwt.uibinder.client.UiBinder;', 'import com.google.gwt.uibinder.client.UiField;', 'import com.google.gwt.uibinder.client.UiHandler;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.Composite;', 'import com.google.gwt.user.client.ui.Label;', 'import com.google.gwt.user.client.ui.VerticalPanel;'], ['import com.google.gwt.http.client.Request;', 'import com.google.gwt.http.client.RequestBuilder;', 'import com.google.gwt.http.client.RequestCallback;', 'import com.google.gwt.http.client.RequestException;', 'import com.google.gwt.http.client.Response;', 'import java.util.UUID;'], ['import com.google.gwt.event.dom.client.ClickEvent;', 'import com.google.gwt.event.dom.client.ClickHandler;', 'import com.google.gwt.event.shared.HandlerRegistration;', 'import com.google.gwt.user.client.DOM;', 'import com.google.gwt.user.client.ui.Composite;', 'import com.google.gwt.user.client.ui.Widget;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.event.dom.client.ClickEvent;', 'import com.google.gwt.event.dom.client.ClickHandler;', 'import com.google.gwt.event.dom.client.KeyCodes;', 'import com.google.gwt.event.dom.client.KeyUpEvent;', 'import com.google.gwt.event.dom.client.KeyUpHandler;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.DialogBox;', 'import com.google.gwt.user.client.ui.HTML;', 'import com.google.gwt.user.client.ui.Label;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.TextBox;', 'import com.google.gwt.user.client.ui.VerticalPanel;'], ['import com.google.gwt.event.dom.client.ClickEvent;', 'import com.google.gwt.event.dom.client.ClickHandler;', 'import com.google.gwt.event.logical.shared.FormHandler;', 'import com.google.gwt.event.logical.shared.FormSubmitCompleteEvent;', 'import com.google.gwt.event.logical.shared.FormSubmitEvent;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.FormPanel;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.TextBox;', 'import com.google.gwt.user.client.ui.VerticalPanel;'], ['import com.google.gwt.core.client.GWT;', 'import com.google.gwt.dom.client.ButtonElement;', 'import com.google.gwt.dom.client.Document;', 'import com.google.gwt.dom.client.NodeList;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.RootPanel;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.http.client.Request;', 'import com.google.gwt.http.client.RequestBuilder;', 'import com.google.gwt.http.client.RequestCallback;', 'import com.google.gwt.http.client.RequestException;', 'import com.google.gwt.http.client.Response;', 'import com.google.gwt.user.client.Window;'], ['import com.google.gwt.jsonp.client.JsonpRequestBuilder;', 'import com.google.gwt.user.client.Window;', 'import com.google.gwt.user.client.rpc.AsyncCallback;'], ['import com.google.gwt.user.client.ui.DialogBox;', 'import com.google.gwt.user.client.ui.Grid;', 'import com.google.gwt.user.client.ui.HTML;', 'import com.google.gwt.user.client.ui.TextBox;'], ['import com.google.gwt.http.client.Request;', 'import com.google.gwt.http.client.RequestBuilder;', 'import com.google.gwt.http.client.RequestCallback;', 'import com.google.gwt.http.client.RequestException;', 'import com.google.gwt.http.client.Response;', 'import com.google.gwt.http.client.URL;'], ['import com.google.gwt.http.client.Request;', 'import com.google.gwt.http.client.RequestBuilder;', 'import com.google.gwt.http.client.RequestCallback;', 'import com.google.gwt.http.client.RequestException;', 'import com.google.gwt.http.client.Response;', 'import com.google.gwt.http.client.URL;', 'import com.google.gwt.user.client.Window;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.CheckBox;', 'import com.google.gwt.user.client.ui.ClickListener;', 'import com.google.gwt.user.client.ui.DockPanel;', 'import com.google.gwt.user.client.ui.Grid;', 'import com.google.gwt.user.client.ui.HTML;', 'import com.google.gwt.user.client.ui.HorizontalPanel;', 'import com.google.gwt.user.client.ui.RadioButton;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.VerticalPanel;', 'import com.google.gwt.user.client.ui.Widget;', 'import java.util.ArrayList;', 'import java.util.Iterator;', 'import java.util.List;'], ['import com.google.gwt.dom.client.DOM;', 'import com.google.gwt.event.dom.client.ClickEvent;', 'import com.google.gwt.event.dom.client.ClickHandler;', 'import com.google.gwt.event.dom.client.SubmitEvent;', 'import com.google.gwt.event.dom.client.SubmitHandler;', 'import com.google.gwt.event.shared.HandlerRegistration;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.FormPanel;'], ['import com.google.gwt.animation.client.Animation;', 'import com.google.gwt.core.client.GWT;', 'import com.google.gwt.dom.client.Element;', 'import com.google.gwt.user.client.ui.FlowPanel;'], ['import com.google.gwt.user.client.ui.AbsolutePanel;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.SimplePanel;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.http.client.Request;', 'import com.google.gwt.http.client.RequestBuilder;', 'import com.google.gwt.http.client.RequestCallback;', 'import com.google.gwt.http.client.RequestException;', 'import com.google.gwt.http.client.Response;', 'import com.google.gwt.http.client.URL;', 'import com.google.gwt.user.client.Window;', 'import com.google.gwt.user.client.ui.HTML;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.Widget;', 'import com.google.gwt.core.client.JsArray;'], ['import com.google.gwt.event.dom.client.HasClickHandlers;', 'import com.google.gwt.user.client.ui.Composite;', 'import com.google.gwt.user.client.ui.Hyperlink;', 'import com.google.gwt.user.client.ui.VerticalPanel;', 'import com.google.gwt.user.client.ui.Widget;'], ['import com.google.gwt.user.server.rpc.RemoteServiceServlet;', 'import javax.servlet.ServletContext;', 'import javax.servlet.ServletOutputStream;', 'import javax.servlet.http.HttpServletResponse;', 'import java.io.DataInputStream;', 'import java.io.File;', 'import java.io.FileInputStream;'], ['import com.google.gwt.user.client.ui.HorizontalPanel;', 'import com.google.gwt.user.client.ui.Label;', 'import com.sencha.gxt.widget.core.client.container.LayoutContainer;'], ['import com.google.gwt.dom.client.Element;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.Composite;', 'import com.google.gwt.user.client.ui.Label;', 'import com.google.gwt.user.client.ui.RootPanel;', 'import com.google.gwt.user.client.ui.VerticalPanel;', 'import com.google.gwt.event.dom.client.ClickEvent;', 'import com.google.gwt.event.dom.client.ClickHandler;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.http.client.Request;', 'import com.google.gwt.http.client.RequestBuilder;', 'import com.google.gwt.http.client.RequestCallback;', 'import com.google.gwt.http.client.RequestException;', 'import com.google.gwt.http.client.Response;', 'import com.google.gwt.http.client.URL;', 'import com.google.gwt.user.client.Window;'], ['import com.google.gwt.event.dom.client.ClickEvent;', 'import com.google.gwt.event.dom.client.ClickHandler;', 'import com.google.gwt.event.dom.client.MouseOutEvent;', 'import com.google.gwt.event.dom.client.MouseOutHandler;', 'import com.google.gwt.user.client.ui.Composite;', 'import com.google.gwt.user.client.ui.HorizontalPanel;', 'import com.google.gwt.user.client.ui.Label;', 'import com.google.gwt.user.client.ui.Widget;'], ['import com.google.gwt.core.client.GWT;', 'import com.google.gwt.core.client.RunAsyncCallback;', 'import com.google.gwt.user.client.rpc.AsyncCallback;', 'import com.google.gwt.user.client.ui.DockLayoutPanel;', 'import com.google.gwt.user.client.ui.Label;', 'import com.google.gwt.user.client.ui.ResizeComposite;', 'import com.google.gwt.dom.client.Style.Unit;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.event.dom.client.ClickEvent;', 'import com.google.gwt.event.dom.client.ClickHandler;', 'import com.google.gwt.user.client.Window;', 'import com.google.gwt.user.client.ui.Button;', 'import com.google.gwt.user.client.ui.Label;', 'import com.google.gwt.user.client.ui.RootPanel;'], ['import com.google.gwt.user.client.ui.VerticalPanel;', 'import com.google.gwt.user.client.ui.DialogBox;', 'import com.google.gwt.user.client.ui.HTML;', 'import com.google.gwt.user.client.ui.RootPanel;'], ['import com.google.gwt.core.client.EntryPoint;', 'import com.google.gwt.dom.client.Style.Unit;', 'import com.google.gwt.user.client.ui.HTML;', 'import com.google.gwt.user.client.ui.LayoutPanel;', 'import com.google.gwt.user.client.ui.RootLayoutPanel;', 'import com.google.gwt.user.client.ui.Widget;'], ['import com.google.gwt.event.logical.shared.BeforeSelectionEvent;', 'import com.google.gwt.event.logical.shared.BeforeSelectionHandler;', 'import com.google.gwt.event.logical.shared.SelectionEvent;', 'import com.google.gwt.event.logical.shared.SelectionHandler;', 'import com.google.gwt.user.client.Window;', 'import com.google.gwt.user.client.ui.TabBar;', 'import com.google.gwt.core.client.EntryPoint;'], ['import com.google.gwt.dom.client.BodyElement;', 'import com.google.gwt.dom.client.Document;', 'import com.google.gwt.dom.client.Element;', 'import com.google.gwt.user.client.DOM;', 'import java.util.ArrayList;'], ['import com.google.gwt.event.dom.client.BlurEvent;', 'import com.google.gwt.event.dom.client.BlurHandler;', 'import com.google.gwt.event.dom.client.ClickEvent;', 'import com.google.gwt.event.dom.client.ClickHandler;', 'import com.google.gwt.event.dom.client.KeyDownEvent;', 'import com.google.gwt.event.dom.client.KeyDownHandler;', 'import com.google.gwt.event.shared.HandlerRegistration;', 'import com.google.gwt.user.client.DOM;', 'import com.google.gwt.user.client.ui.ComplexPanel;', 'import com.google.gwt.user.client.ui.HasBlurHandlers;', 'import com.google.gwt.user.client.ui.HasClickHandlers;', 'import com.google.gwt.user.client.ui.HasHTML;', 'import com.google.gwt.user.client.ui.HasKeyDownHandlers;', 'import com.google.gwt.user.client.ui.HasText;', 'import com.google.gwt.user.client.ui.Widget;']]
import_list = predicted_imports
append_imports(save_directory, code_name, codes, import_list)

In [ ]:
save_directory = "/home/azmain/Code for Compilation Test/one-shot/xstream"
code_name = "xstream"
codes = xstream_codes
predicted_imports = [['import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.converters.MarshallingContext;', 'import com.thoughtworks.xstream.converters.UnmarshallingContext;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;'], ['import java.io.BufferedReader;', 'import java.io.FileReader;', 'import java.io.IOException;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.DomDriver;'], ['import java.util.ArrayList;', 'import java.util.List;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.json.JettisonMappedXmlDriver;'], ['import java.util.ArrayList;', 'import java.util.List;', 'import com.thoughtworks.xstream.XStream;'], ['import java.io.FileNotFoundException;', 'import java.io.PrintWriter;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.DomDriver;'], ['import java.io.File;', 'import java.io.FileOutputStream;', 'import java.io.OutputStream;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.DomDriver;'], ['import com.thoughtworks.xstream.annotations.XStreamAsAttribute;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;', 'import org.ksoap2.SoapEnvelope;', 'import org.ksoap2.serialization.SoapObject;', 'import org.ksoap2.serialization.SoapSerializationEnvelope;', 'import org.ksoap2.transport.HttpTransportSE;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.DomDriver;'], ['import java.io.FileInputStream;', 'import java.io.FileNotFoundException;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.DomDriver;'], ['import java.io.StringWriter;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.PrettyPrintWriter;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.converters.collections.CollectionConverter;', 'import com.thoughtworks.xstream.mapper.ClassAliasingMapper;', 'import java.util.ArrayList;', 'import java.util.List;'], ['import java.util.List;', 'import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.converters.MarshallingContext;', 'import com.thoughtworks.xstream.converters.UnmarshallingContext;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;'], ['import com.thoughtworks.xstream.mapper.ClassAliasingMapper;', 'import com.thoughtworks.xstream.mapper.Mapper;'], ['import com.thoughtworks.xstream.converters.SingleValueConverter;', 'import com.thoughtworks.xstream.converters.enums.AbstractSingleValueConverter;'], ['import java.util.HashSet;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.converters.CollectionConverter;', 'import com.thoughtworks.xstream.converters.MarshallingContext;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.mapper.MapperWrapper;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.annotations.XStreamAlias;', 'import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;', 'import com.thoughtworks.xstream.io.MarshallingContext;', 'import com.thoughtworks.xstream.io.UnmarshallingContext;'], ['import java.util.HashMap;', 'import java.util.Map;', 'import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.converters.MarshallingContext;', 'import com.thoughtworks.xstream.converters.UnmarshallingContext;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;'], ['import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.converters.MarshallingContext;', 'import com.thoughtworks.xstream.converters.UnmarshallingContext;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;'], ['import java.io.File;', 'import java.io.FileInputStream;', 'import java.io.InputStream;', 'import java.io.InputStreamReader;', 'import java.io.Reader;', 'import java.nio.charset.Charset;', 'import com.thoughtworks.xstream.XStream;'], ['import java.util.Map;', 'import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.converters.MarshallingContext;', 'import com.thoughtworks.xstream.converters.UnmarshallingContext;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;', 'import com.thoughtworks.xstream.mapper.Mapper;', 'import com.thoughtworks.xstream.mapper.ReflectionProvider;', 'import com.thoughtworks.xstream.converters.reflection.ReflectionConverter;'], ['import java.io.File;', 'import java.util.Arrays;', 'import java.util.List;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.annotations.XStreamAlias;', 'import com.thoughtworks.xstream.annotations.XStreamConverter;', 'import com.thoughtworks.xstream.converters.extended.ToAttributedValueConverter;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.annotations.XStreamAlias;', 'import com.thoughtworks.xstream.io.xml.XppDriver;', 'import com.thoughtworks.xstream.converters.extended.JavaBeanConverter;'], ['import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;', 'import com.thoughtworks.xstream.mapper.Mapper;', 'import com.thoughtworks.xstream.converters.reflection.ReflectionProvider;', 'import com.thoughtworks.xstream.converters.reflection.ReflectionConverter;', 'import com.thoughtworks.xstream.converters.UnmarshallingContext;', 'import com.thoughtworks.xstream.converters.MarshallingContext;'], ['import com.thoughtworks.xstream.annotations.XStreamAlias;', 'import com.thoughtworks.xstream.annotations.XStreamConverter;', 'import com.thoughtworks.xstream.converters.extended.ToAttributedValueConverter;'], ['import java.util.ArrayList;', 'import java.util.Collection;', 'import java.util.List;', 'import com.thoughtworks.xstream.converters.SingleValueConverter;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.converters.MarshallingContext;', 'import com.thoughtworks.xstream.converters.UnmarshallingContext;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;', 'import org.apache.commons.lang3.text.WordUtils;', 'import xstream.Content;'], ['import java.util.HashMap;', 'import java.util.Map;', 'import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.converters.MarshallingContext;', 'import com.thoughtworks.xstream.converters.UnmarshallingContext;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;'], ['import java.util.ArrayList;', 'import java.util.List;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.DomDriver;'], ['import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;', 'import com.thoughtworks.xstream.converters.MarshallingContext;', 'import com.thoughtworks.xstream.converters.UnmarshallingContext;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.annotations.XStreamAlias;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.mapper.MapperWrapper;', 'import com.thoughtworks.xstream.hibernate.converter.HibernateMapper;', 'import com.thoughtworks.xstream.hibernate.converter.HibernateProxyConverter;', 'import com.thoughtworks.xstream.hibernate.converter.HibernatePersistentCollectionConverter;', 'import com.thoughtworks.xstream.hibernate.converter.HibernatePersistentMapConverter;', 'import com.thoughtworks.xstream.hibernate.converter.HibernatePersistentSortedMapConverter;', 'import com.thoughtworks.xstream.hibernate.converter.HibernatePersistentSortedSetConverter;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.DomDriver;', 'import com.thoughtworks.xstream.mapper.Mapper;', 'import xstream.ActionMessage;', 'import xstream.MessageHeader;', 'import xstream.MessageBody;', 'import xstream.RemotingMessage;', 'import xstream.CommandMessage;', 'import xstream.AcknowledgeMessage;', 'import xstream.ErrorMessage;', 'import xstream.ASObject;', 'import xstream.AsyncMessage;', 'import xstream.CommandMessageExt;', 'import xstream.AcknowledgeMessageExt;'], ['import java.util.List;', 'import java.util.ArrayList;', 'import java.util.Iterator;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.annotations.XStreamAlias;', 'import com.thoughtworks.xstream.annotations.XStreamImplicit;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.annotations.XStreamAlias;', 'import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.converters.SingleValueConverter;', 'import com.thoughtworks.xstream.mapper.Mapper;', 'import com.thoughtworks.xstream.mapper.MapperWrapper;', 'import com.thoughtworks.xstream.converters.ConverterLookup;', 'import com.thoughtworks.xstream.converters.reflection.ReflectionProvider;', 'import org.junit.Test;'], ['import java.util.Map;', 'import java.util.HashMap;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.StaxDriver;'], ['import java.util.ArrayList;', 'import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.converters.MarshallingContext;', 'import com.thoughtworks.xstream.converters.UnmarshallingContext;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;'], ['import java.util.ArrayList;', 'import java.util.List;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.annotations.XStreamAlias;'], ['import java.io.Writer;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;', 'import com.thoughtworks.xstream.io.WriterWrapper;', 'import com.thoughtworks.xstream.io.xml.PrettyPrintWriter;', 'import com.thoughtworks.xstream.io.xml.XppDriver;'], ['import com.thoughtworks.xstream.converters.Converter;', 'import com.thoughtworks.xstream.io.HierarchicalStreamReader;', 'import com.thoughtworks.xstream.io.HierarchicalStreamWriter;', 'import com.thoughtworks.xstream.io.MarshallingContext;', 'import com.thoughtworks.xstream.io.UnmarshallingContext;'], ['import java.util.List;', 'import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.io.xml.StaxDriver;', 'import java.util.ArrayList;'], ['import java.util.List;', 'import com.thoughtworks.xstream.annotations.XStreamAlias;', 'import com.thoughtworks.xstream.annotations.XStreamImplicit;'], ['import com.thoughtworks.xstream.XStream;', 'import com.thoughtworks.xstream.annotations.XStreamAlias;', 'import com.thoughtworks.xstream.io.xml.DomDriver;', 'import java.util.List;', 'import java.util.ArrayList;'], ['import java.io.File;', 'import java.io.FileNotFoundException;', 'import java.io.FileOutputStream;', 'import com.thoughtworks.xstream.XStream;']]
import_list = predicted_imports
append_imports(save_directory, code_name, codes, import_list)

# One-shot Self-Consistency with Semantically Similar Example

In [ ]:
def get_modified_temp_prediction(example, code_snippet):
    for example_code, correct_output in example:
        expected_output = '\n'.join(correct_output)
        
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Reply with to-the-point answer, no elaboration."},
            {"role": "system", "name":"example_user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{example_code}\n\"\"\""},
            {"role": "system", "name": "example_assistant", "content": f"{expected_output}"},
            {"role": "user", "content": f"Do not check for any import statements in the code. Only give correct imports by not using wildcard imports. Please note that you need to pay close attention and your response should be specific and accurate. Avoid repetition and must not generate wrong and nonexistent imports:\n\"\"\"\n{code_snippet}\n\"\"\""},
        ],
        temperature=1,
        top_p=1,
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
from collections import Counter

def get_consistent_item(k_items_list, code_snippet, one_shot_semantic_example):
    
    # Count the frequency of each set of sublists
    counts = Counter(frozenset(sublist) for sublist in k_items_list)

    # Find the maximum frequency count
    max_count = max(counts.values())

    # Find all sets of sublists with the maximum frequency count
    most_common_sublists_set = {frozenset(sublist) for sublist in counts if counts[sublist] == max_count}

    # Convert the sets of sublists back to lists
    most_common_sublists = [list(sublist) for sublist in most_common_sublists_set]

    # Find the number of sublists that have the highest frequency count
    max_consistency_count = len(most_common_sublists)
    
    max_check_time = 5
    i = 0
    while max_consistency_count!=1:
        if i > max_check_time:
            return []
        time.sleep(2)
        new_predicted_output = re.findall(r"import\s+[\w\., ]+;", get_modified_temp_prediction(one_shot_semantic_example, code_snippet))
        k_items_list.append(new_predicted_output)
        counts = Counter(frozenset(sublist) for sublist in k_items_list)
        max_count = max(counts.values())
        most_common_sublists_set = {frozenset(sublist) for sublist in counts if counts[sublist] == max_count}
        most_common_sublists = [list(sublist) for sublist in most_common_sublists_set]
        max_consistency_count = len(most_common_sublists)
        i+=1

    consistent_item = most_common_sublists[0]
    
    return consistent_item

In [ ]:
def get_self_consistent_predictions(semantic_example, dataset):
    y_pred_list = []
    idx = 0
    for code_snippet, correct_imports in tqdm(dataset):
        one_shot_semantic_example = [semantic_example[idx]]
        k_output_sample = []
        k = 10
        for i in range(0, k):
            time.sleep(2)
            predicted_output = re.findall(r"import\s+[\w\., ]+;", get_modified_temp_prediction(one_shot_semantic_example, code_snippet))
            k_output_sample.append(predicted_output)
        y_pred_list.append(get_consistent_item(k_output_sample, code_snippet, one_shot_semantic_example))
        idx = idx + 1
    return y_pred_list

In [ ]:
# Prediction with Semantic One-shot Self-Consistency and Modified Parameters for Android Classes

print("\nPrediction with Semantic One-shot Self-Consistency and Modified Parameters for Android Classes:\n")
android_semantic_list = get_semantic_similarity(android_dataset)
y_pred = get_self_consistent_predictions(android_semantic_list, android_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", android_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, android_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Semantic One-shot Self-Consistency and Modified Parameters for JDK Classes

print("\nPrediction with Semantic One-shot Self-Consistency and Modified Parameters for JDK Classes:\n")
jdk_semantic_list = get_semantic_similarity(jdk_dataset)
y_pred = get_self_consistent_predictions(jdk_semantic_list, jdk_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jdk_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jdk_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Semantic One-shot Self-Consistency and Modified Parameters for Hibernate Classes

print("\nPrediction with Semantic One-shot Self-Consistency and Modified Parameters for Hibernate Classes:\n")
hibernate_semantic_list = get_semantic_similarity(hibernate_dataset)
y_pred = get_self_consistent_predictions(hibernate_semantic_list, hibernate_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", hibernate_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, hibernate_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Semantic One-shot Self-Consistency and Modified Parameters for Joda-Time Classes

print("\nPrediction with Semantic One-shot Self-Consistency and Modified Parameters for Joda-Time Classes:\n")
jodatime_semantic_list = get_semantic_similarity(jodatime_dataset)
y_pred = get_self_consistent_predictions(jodatime_semantic_list, jodatime_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", jodatime_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, jodatime_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Semantic One-shot Self-Consistency and Modified Parameters for GWT Classes

print("\nPrediction with Semantic One-shot Self-Consistency and Modified Parameters for GWT Classes:\n")
gwt_semantic_list = get_semantic_similarity(gwt_dataset)
y_pred = get_self_consistent_predictions(gwt_semantic_list, gwt_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", gwt_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, gwt_y_true)
eval_performance(y_pred_processed, y_true_processed)

In [ ]:
# Prediction with Semantic One-shot Self-Consistency and Modified Parameters for XStream Classes

print("\nPrediction with Semantic One-shot Self-Consistency and Modified Parameters for XStream Classes:\n")
xstream_semantic_list = get_semantic_similarity(xstream_dataset)
y_pred = get_self_consistent_predictions(xstream_semantic_list, xstream_test_examples)
print("\nPredicted Import List:", y_pred)
print("\nCorrect Import List:", xstream_y_true)
y_pred_processed, y_true_processed = pred_process(y_pred, xstream_y_true)
eval_performance(y_pred_processed, y_true_processed)